In [1]:
import os, sys
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")
import traci
import numpy as np
import pandas as pd
import gtsam
import gtsam_unstable

In [2]:
sumoBinary = "/usr/local/bin/sumo"
sumoCmd = [sumoBinary, "-c", "cross.sumocfg"]

seed = 1
randomState = np.random.RandomState(seed)

In [3]:
def printInfo():
    print(F"current time: {traci.simulation.getTime()}")
    print(F"current vehicle list: {traci.vehicle.getIDList()}")
    print(F"departed vehicle list: {traci.simulation.getDepartedIDList()}")
    print(F"arrived vehicle list: {traci.simulation.getArrivedIDList()}")
    ids = traci.vehicle.getIDList()
    for x in ids:
        pos = traci.vehicle.getPosition(x)
        angle = traci.vehicle.getAngle(x)
        print(F"{x}: x:{pos[0]}, y:{pos[1]}, angle:{angle / 180 * np.pi}")
    print("\n")

In [4]:
max_range = 20

gps_sigma_x, gps_sigma_y, gps_sigma_angle = 5, 5, np.pi * 20 / 180
odom_sigma_x, odom_sigma_y, odom_sigma_angle = gps_sigma_x / 10, gps_sigma_y / 10, gps_sigma_angle / 10
relative_sigma_x, relative_sigma_y, relative_sigma_angle = gps_sigma_x / 10, gps_sigma_y / 10, gps_sigma_angle / 10

gps_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array(
                        [gps_sigma_x, gps_sigma_y, gps_sigma_angle]))
odom_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array(
                        [odom_sigma_x, odom_sigma_y, odom_sigma_angle]))
relative_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array(
                        [relative_sigma_x, relative_sigma_y, relative_sigma_angle]))

In [5]:
lag = 10.0
smoother = gtsam_unstable.BatchFixedLagSmoother(lag)

new_factors = gtsam.NonlinearFactorGraph()
new_values = gtsam.Values()
new_timestamps = gtsam_unstable.FixedLagSmootherKeyTimestampMap()
name = {}

In [6]:
def symbol(x, t):
    return (name[x] << 15) + t

def pose(x):
    pos = traci.vehicle.getPosition(x)
    angle = traci.vehicle.getAngle(x) / 180 * np.pi
    return gtsam.Pose2(pos[0], pos[1], angle)

def noise_gps_measurement(x):
    current_pose = pose(x)
    noise_x = randomState.normal(0.0, gps_sigma_x)
    noise_y = randomState.normal(0.0, gps_sigma_y)
    noise_angle = randomState.normal(0.0, gps_sigma_angle)
    gps_measurement = current_pose.compose(gtsam.Pose2(
        noise_x, noise_y, noise_angle
    ))
    return gps_measurement

previous_states = {}
current_states = {}
def noise_odom_measurement(x):
    previous_pose = previous_states[x]
    current_pose = current_states[x]
    odom_measurement = previous_pose.between(current_pose)
    noise_x = randomState.normal(0.0, odom_sigma_x)
    noise_y = randomState.normal(0.0, odom_sigma_y)
    noise_angle = randomState.normal(0.0, odom_sigma_angle)
    odom_measurement = odom_measurement.compose(gtsam.Pose2(
        noise_x, noise_y, noise_angle
    ))
    return odom_measurement

def noise_relative_measurement(x, y):
    relative_measurement = pose(x).between(pose(y))
    noise_x = randomState.normal(0.0, relative_sigma_x)
    noise_y = randomState.normal(0.0, relative_sigma_y)
    noise_angle = randomState.normal(0.0, relative_sigma_angle)
    relative_measurement = relative_measurement.compose(gtsam.Pose2(
        noise_x, noise_y, noise_angle
    ))
    return relative_measurement

def euclidean_distance(x, y):
    pos_x = np.array(traci.vehicle.getPosition(x))
    pos_y = np.array(traci.vehicle.getPosition(y))
    return np.linalg.norm(pos_x - pos_y)

In [7]:
traci.start(sumoCmd)
data = []
time = 0
while time <= 63:
    print(F"time: {time}")
    if not traci.vehicle.getIDCount():
        traci.simulationStep()
        time += 1 
        continue
    printInfo()
    ids = traci.vehicle.getIDList()
    for x in ids:
        if x not in name:
            name[x] = len(name)

        current_key = symbol(x, time)
        new_timestamps.insert((current_key, time))

        # absolute measurement
        gps_measurement = noise_gps_measurement(x)
        new_factors.push_back(gtsam.PriorFactorPose2(
            current_key, gps_measurement, gps_noise
        ))
        new_values.insert(current_key, gps_measurement)
        
        current_states[x] = gps_measurement
        
        # temperal relative measurement
        if x in previous_states:
            previous_key = symbol(x, time - 1)
            odom_measurement = noise_odom_measurement(x)
            new_factors.push_back(gtsam.BetweenFactorPose2(
                previous_key, current_key, odom_measurement, odom_noise 
            ))

    # spatial relative measurement
    for i in ids:
        for j in ids:
            if i == j or euclidean_distance(i, j) > max_range:
                continue
            relative_measurement = noise_relative_measurement(i, j)
            new_factors.push_back(gtsam.BetweenFactorPose2(
                symbol(i, time), symbol(j, time), relative_measurement, relative_noise
            ))
            
    smoother.update(new_factors, new_values, new_timestamps)
    print("after optimization:")
    for x in ids:
        current_key = symbol(x, time)
        estimation = smoother.calculateEstimatePose2(current_key)
        groundtruth = pose(x)
        difference = groundtruth.between(estimation)
        print("Timestamp = " + str(time) + ", Vehicle = " + x)
        print(F"Estimation: {estimation}")
        print(F"GroundTruth: {groundtruth}")
        print(F"Difference: {groundtruth.x() - estimation.x(), groundtruth.y() - estimation.y(), difference.theta()}")
        value = {
            "timestamp": time, "vehicle": x, 
            "groundtruth_x":groundtruth.x(),
            "groundtruth_y":groundtruth.y(),
            "groundtruth_theta":groundtruth.theta(),
            "estimation_x":estimation.x(),
            "estimation_y":estimation.y(),
            "estimation_theta":estimation.theta(),
            "difference_x":groundtruth.x() - estimation.x(),
            "difference_y":groundtruth.y() - estimation.y(),
            "difference_theta":difference.theta(),
        }
        data.append(value)
    print('\n')

    print(F"end time {time}\n")

    new_timestamps.clear()
    new_values.clear()
    new_factors.resize(0)
    previous_states = current_states
    current_states = {}
            
    traci.simulationStep()
    time += 1 
df = pd.DataFrame(data)
df.to_csv("cooperative-localization-result.csv", index=False)
traci.close()

 Retrying in 1 seconds
time: 0
time: 1
time: 2
time: 3
current time: 3.0
current vehicle list: ('DU.0',)
departed vehicle list: ('DU.0',)
arrived vehicle list: ()
DU.0: x:92.0, y:194.9, angle:3.141592653589793


after optimization:
Timestamp = 3, Vehicle = DU.0
Estimation: (83.8783, 197.959, 2.95723)

GroundTruth: (92, 194.9, 3.14159)

Difference: (8.121726818316205, -3.058782068250366, -0.1843667218605023)


end time 3

time: 4
current time: 4.0
current vehicle list: ('DU.0',)
departed vehicle list: ()
arrived vehicle list: ()
DU.0: x:92.0, y:192.75685595781542, angle:3.141592653589793


after optimization:
Timestamp = 4, Vehicle = DU.0
Estimation: (97.1459, 188.793, 2.33694)

GroundTruth: (92, 192.757, 3.14159)

Difference: (-5.145925844957162, 3.9641324318155, -0.8046570921904579)


end time 4

time: 5
current time: 5.0
current vehicle list: ('DU.0',)
departed vehicle list: ()
arrived vehicle list: ()
DU.0: x:92.0, y:189.20650960281492, angle:3.141592653589793


after optimization:


after optimization:
Timestamp = 22, Vehicle = DU.0
Estimation: (85.4087, 33.0014, -2.85155)

GroundTruth: (92, 30.0195, 3.14159)

Difference: (6.5913198978308145, -2.9818978274574732, 0.2900395076078166)
Timestamp = 22, Vehicle = DU.1
Estimation: (92.1966, 72.2668, -3.08663)

GroundTruth: (92, 73.7706, 3.14159)

Difference: (-0.19656969621794929, 1.5037680566945397, 0.05496611675007876)
Timestamp = 22, Vehicle = DU.2
Estimation: (91.1776, 106.963, -3.10074)

GroundTruth: (92, 108.86, 3.14159)

Difference: (0.8224183851669267, 1.8967815493055156, 0.04084785947600337)
Timestamp = 22, Vehicle = DU.3
Estimation: (88.7741, 158.278, -2.53096)

GroundTruth: (92, 165.466, 3.14159)

Difference: (3.2258715600787013, 7.188728842054047, 0.6106355086963723)
Timestamp = 22, Vehicle = DU.4
Estimation: (93.4236, 194.974, -2.8974)

GroundTruth: (92, 192.564, 3.14159)

Difference: (-1.423571022471279, -2.4091295346889012, 0.24418791057890946)
Timestamp = 22, Vehicle = LR.0
Estimation: (83.7396, 90.9479,

Timestamp = 29, Vehicle = UD.1
Estimation: (110.073, 75.4853, -0.710341)

GroundTruth: (108, 78.6832, 0)

Difference: (-2.072681161990232, 3.1979296808743527, -0.7103406036100239)
Timestamp = 29, Vehicle = UD.2
Estimation: (108.199, 43.9874, 0.0699203)

GroundTruth: (108, 44.5263, 0)

Difference: (-0.19909632978581726, 0.5389256952624066, 0.06992027305705809)
Timestamp = 29, Vehicle = UD.3
Estimation: (114.415, 18.4718, 0.512597)

GroundTruth: (108, 10.2015, 0)

Difference: (-6.414902032442328, -8.270368799210058, 0.5125965792154148)


end time 29

time: 30
current time: 30.0
current vehicle list: ('DU.1', 'DU.2', 'DU.3', 'DU.4', 'LR.0', 'LR.1', 'RL.0', 'UD.0', 'UD.1', 'UD.2', 'UD.3', 'UD.4')
departed vehicle list: ('UD.4',)
arrived vehicle list: ()
DU.1: x:92.0, y:3.437001212849566, angle:3.141592653589793
DU.2: x:92.0, y:32.651848813208716, angle:3.141592653589793
DU.3: x:92.0, y:88.09012522430176, angle:3.141592653589793
DU.4: x:92.0, y:133.08936935739084, angle:3.141592653589793
LR

Timestamp = 35, Vehicle = LR.0
Estimation: (95.4448, 85.4594, 1.92222)

GroundTruth: (85.399, 92, 1.5708)

Difference: (-10.045807795203615, 6.540629628536664, 0.3514257444493958)
Timestamp = 35, Vehicle = LR.1
Estimation: (95.2569, 89.6891, 1.74904)

GroundTruth: (85.3989, 95.2, 1.5708)

Difference: (-9.858038310686254, 5.510850281971003, 0.1782422485561147)
Timestamp = 35, Vehicle = RL.0
Estimation: (115.407, 110.656, -1.1084)

GroundTruth: (114.602, 108, -1.5708)

Difference: (-0.8050508069706979, -2.6561771576643736, 0.46239584347649704)
Timestamp = 35, Vehicle = UD.0
Estimation: (94.1489, 186.566, -0.450535)

GroundTruth: (108, 188.921, 0)

Difference: (13.851125497309354, 2.355285499257832, -0.4505352896611108)
Timestamp = 35, Vehicle = UD.1
Estimation: (104.581, 124.231, 0.124135)

GroundTruth: (108, 128.453, 0)

Difference: (3.419049775106302, 4.221989457433722, 0.1241345194637228)
Timestamp = 35, Vehicle = UD.2
Estimation: (116.295, 103.813, 0.230374)

GroundTruth: (108, 105.4

after optimization:
Timestamp = 43, Vehicle = DU.4
Estimation: (83.9759, 19.8814, 2.71714)

GroundTruth: (92, 29.4629, 3.14159)

Difference: (8.024124558520015, 9.581505367329225, -0.4244567584226648)
Timestamp = 43, Vehicle = LR.0
Estimation: (82.1108, 89.3205, 1.88904)

GroundTruth: (85.399, 92, 1.5708)

Difference: (3.2881894246845036, 2.679466900951226, 0.3182468956468983)
Timestamp = 43, Vehicle = LR.1
Estimation: (81.4347, 94.0023, 1.9658)

GroundTruth: (85.3989, 95.2, 1.5708)

Difference: (3.9642021959874683, 1.197668042301828, 0.3950067846601828)
Timestamp = 43, Vehicle = RL.0
Estimation: (114.446, 107.811, -1.50147)

GroundTruth: (114.601, 108, -1.5708)

Difference: (0.15511505884636279, 0.18872866039994562, 0.0693260851943202)
Timestamp = 43, Vehicle = UD.1
Estimation: (95.0682, 188.354, 0.178001)

GroundTruth: (108, 195.38, 0)

Difference: (12.93179324393374, 7.025828426456883, 0.17800058666536664)
Timestamp = 43, Vehicle = UD.2
Estimation: (97.0552, 168.899, 0.118534)

Grou

after optimization:
Timestamp = 60, Vehicle = LR.1
Estimation: (194.387, 102.071, 1.4877)

GroundTruth: (190.745, 95.2, 1.5708)

Difference: (-3.6427766214983137, -6.871499706746491, -0.083091796088811)


end time 60

time: 61
current time: 61.0
current vehicle list: ('LR.1',)
departed vehicle list: ()
arrived vehicle list: ()
LR.1: x:198.29013951302736, y:95.2, angle:1.5707963267948966


after optimization:
Timestamp = 61, Vehicle = LR.1
Estimation: (199.589, 103.178, 1.36927)

GroundTruth: (198.29, 95.2, 1.5708)

Difference: (-1.298985002716563, -7.977566732207023, -0.2015231198584849)


end time 61

time: 62
time: 63


In [8]:
df

,timestamp,vehicle,groundtruth_x,groundtruth_y,groundtruth_theta,estimation_x,estimation_y,estimation_theta,difference_x,difference_y,difference_theta
0,3,DU.0,92.000000,194.900000,3.141593,83.878273,197.958782,2.957226,8.121727,-3.058782,-0.184367
1,4,DU.0,92.000000,192.756856,3.141593,97.145926,188.792724,2.336936,-5.145926,3.964132,-0.804657
2,5,DU.0,92.000000,189.206510,3.141593,93.185475,182.147225,2.450031,-1.185475,7.059285,-0.691562
3,6,DU.0,92.000000,184.120272,3.141593,94.994415,184.365831,2.958906,-2.994415,-0.245560,-0.182687
4,6,DU.1,92.000000,194.900000,3.141593,96.509249,195.121972,2.951141,-4.509249,-0.221972,-0.190452
...,...,...,...,...,...,...,...,...,...,...,...
366,58,RL.0,13.540274,108.000000,-1.570796,8.707201,103.082547,-1.264210,4.833073,4.917453,0.306586
367,59,LR.1,182.795250,95.200000,1.570796,193.668505,94.509673,2.025627,-10.873255,0.690327,0.454830
368,59,RL.0,5.173696,108.000000,-1.570796,3.981196,114.101551,-1.434898,1.192500,-6.101551,0.135898
369,60,LR.1,190.744608,95.200000,1.570796,194.387384,102.071500,1.487705,-3.642777,-6.871500,-0.083092


In [9]:
error_x = np.sqrt(np.mean(df["difference_x"]**2))
error_y = np.sqrt(np.mean(df["difference_y"]**2))
error_theta = np.sqrt(np.mean(df["difference_theta"]**2))
print(error_x, error_y, error_theta / np.pi * 180)

6.042559259321393 5.265465454770983 19.784384242028636
